In [2]:
from tmilib import *
from h2o_utils import *

import csv
import h2o
h2o.init()

/afs/.ir.stanford.edu/users/g/k/gkovacs/ve/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/afs/.ir.stanford.edu/users/g/k/gkovacs/ve/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/afs/.ir.stanford.edu/users/g/k/gkovacs/ve/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/afs/.ir.stanford.edu/users/g/k/gkovacs/ve/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator inst

H2O cluster uptime:,2 hours 3 minutes 47 seconds 18 milliseconds
H2O cluster version:,3.8.1.4
H2O cluster name:,gkovacs
H2O cluster total nodes:,1
H2O cluster total free memory:,35.39 GB
H2O cluster total cores:,48
H2O cluster allowed cores:,16
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [4]:
def sum_confusion_matrix_diagonal_v1(confusion_matrix):
  total_correct = 0
  column_idx = 0
  for column_name in confusion_matrix.col_header:
    if column_name == 'Error' or column_name == 'Rate' or column_name == '':
      continue
    total_correct += confusion_matrix[column_name][column_idx]
    column_idx += 1
  return total_correct

def get_correct_ratio_for_classifier(classifier):
  confusion_matrix = classifier._get_metrics(classifier, False, True, False).values()[0].confusion_matrix()
  if 'table' in dir(confusion_matrix):
    confusion_matrix = confusion_matrix.table
  total_correct = sum_confusion_matrix_diagonal_v1(confusion_matrix)
  total_items = 13836775
  return float(total_correct)/total_items


In [5]:
#classifier = load_h2o_model('domainclass_cpn_v89_randomforest_v1.h2o')
#print get_correct_ratio_for_classifier(classifier)
#print classifier

In [5]:
def make_predictions_and_save_for_classifier_num(classifier_num):
  make_predictions_and_save(
    'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1.h2o',
    'domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '.csv',
    'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_predictions.csv',
    4
  )
  clear_h2o_memory()



In [3]:
def make_predictions_and_save_for_classifier_num_all_insession(classifier_num):
  make_predictions_and_save(
    'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1.h2o',
    'domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '_all_insession.csv',
    'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_all_insession_predictions.csv',
    4
  )
  clear_h2o_memory()


In [5]:
make_predictions_and_save_for_classifier_num_all_insession(78)


Parse Progress: [##################################################] 100%

drf prediction Progress: [##################################################] 100%


In [4]:
for classifier_num in [89, 88, 87, 86, 85, 84, 81, 80, 79, 78, 77]:
  make_predictions_and_save_for_classifier_num(classifier_num)

already exists domainclass_cpn_v89_randomforest_v1_predictions.csv
already exists domainclass_cpn_v88_randomforest_v1_predictions.csv
already exists domainclass_cpn_v87_randomforest_v1_predictions.csv
already exists domainclass_cpn_v86_randomforest_v1_predictions.csv
already exists domainclass_cpn_v85_randomforest_v1_predictions.csv
already exists domainclass_cpn_v84_randomforest_v1_predictions.csv
already exists domainclass_cpn_v81_randomforest_v1_predictions.csv
already exists domainclass_cpn_v80_randomforest_v1_predictions.csv
already exists domainclass_cpn_v79_randomforest_v1_predictions.csv
already exists domainclass_cpn_v78_randomforest_v1_predictions.csv
already exists domainclass_cpn_v77_randomforest_v1_predictions.csv


In [9]:
#classifier_num = 89
#predictions = load_h2o_data('domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_predictions.csv')
  


Parse Progress: [##################################################] 100%


In [10]:
#test_data = load_h2o_data('domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '.csv')


Parse Progress: [##################################################] 100%


In [19]:
from itertools import izip

In [22]:
@jsonmemoized
def get_user_to_time_to_predicted_domain_id(all_insession=False):
  if not all_insession:
    predictions_csv_file = 'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_predictions.csv'
  else:
    predictions_csv_file = 'domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_predictions_all_insession.csv'
  if not all_insession:
    test_data_csv_file = 'domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '.csv'
  else:
    test_data_csv_file = 'domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '_all_insession.csv'
  classifier_num = 78
  predictions_csv = csv.reader(sdir_open(predictions_csv_file))
  predictions_header = next(predictions_csv)
  assert predictions_header[0] == 'predict'

  test_data_csv = csv.reader(sdir_open(test_data_csv_file))
  test_data_header = next(test_data_csv)
  assert test_data_header[0] == 'time_sec'
  assert test_data_header[1] == 'user'
  assert test_data_header[2] == 'ref_domain'
  assert test_data_header[3] == 'label'

  user_to_time_to_domain_id = {}
  user_to_time_to_nextdomain_id = {}
  user_to_time_to_prev7_domains_id = {}
  user_to_time_to_immediate_prev7_domains_id = {}

  total_correct = 0
  total_items = 0
  
  output = {}

  for predictions_line,test_data_line in izip(predictions_csv, test_data_csv):
    total_items += 1
    predict = predictions_line[0]
    label = test_data_line[3]
    if predict == label:
      total_correct += 1
      continue
    time_sec = test_data_line[0] # needs to remain string so we can index into things
    user = test_data_line[1]
    if user not in output:
      output[user] = {}
    ref_domain = test_data_line[2]
    ref_domain_id = domain_to_id(ref_domain)
    time_to_domain_id = user_to_time_to_domain_id.get(user, None)
    if time_to_domain_id == None:
      time_to_domain_id = get_recent_domain_id_at_seconds_for_user(user)
      user_to_time_to_domain_id[user] = time_to_domain_id
    time_to_nextdomain_id = user_to_time_to_nextdomain_id.get(user, None)
    if time_to_nextdomain_id == None:
      time_to_nextdomain_id = get_next_domain_id_at_seconds_for_user(user)
      user_to_time_to_nextdomain_id[user] = time_to_nextdomain_id
    time_to_prev7_domains_id = user_to_time_to_prev7_domains_id.get(user, None)
    if time_to_prev7_domains_id == None:
      time_to_prev7_domains_id = get_prev7_domains_id_at_seconds_for_user(user)
      user_to_time_to_prev7_domains_id[user] = time_to_prev7_domains_id
    time_to_immediate_prev7_domains_id = user_to_time_to_prev7_domains_id.get(user, None)
    if time_to_immediate_prev7_domains_id == None:
      time_to_immediate_prev7_domains_id = get_immediate_prev7_domains_id_at_seconds_for_user(user)
      user_to_time_to_immediate_prev7_domains_id[user] = time_to_immediate_prev7_domains_id
    pred_domain_id = -1
    if predict == 'c':
      pred_domain_id = time_to_domain_id[time_sec]
    elif predict == 'n':
      pred_domain_id = time_to_nextdomain_id[time_sec]
    elif predict[0] == 'p':
      num = int(predict[1])
      pred_domain_id = time_to_prev7_domains_id[time_sec][num]
    elif predict[0] == 'i':
      num = int(predict[1])
      pred_domain_id = time_to_immediate_prev7_domains_id[time_sec][num]
    if pred_domain_id == -1:
      pred_domain_id = time_to_domain_id[time_sec]
    if pred_domain_id == ref_domain_id:
      total_correct += 1
    output[user][time_sec] = pred_domain_id
  return output
  

In [6]:
_ = get_user_to_time_to_predicted_domain_id()

In [13]:
@jsonmemoized
def get_user_to_domain_id_to_total_time_spent_predicted():
  output = {}
  user_to_time_to_domain_id = get_user_to_time_to_predicted_domain_id()
  for user,time_to_domain_id in user_to_time_to_domain_id.viewitems():
    output[user] = Counter()
    for time,domain_id in time_to_domain_id.viewitems():
      output[user][domain_id] += 1
  return output

In [10]:
_ = get_user_to_domain_id_to_total_time_spent_predicted()

In [15]:
@jsonmemoized
def get_user_to_domain_to_total_time_spent_predicted():
  output = {}
  user_to_domain_id_to_total_time_spent = get_user_to_domain_id_to_total_time_spent_predicted()
  for user,domain_id_to_total_time_spent in user_to_domain_id_to_total_time_spent.viewitems():
    output[user] = {}
    for domain_id,total_time_spent in domain_id_to_total_time_spent.viewitems():
      domain = id_to_domain(int(domain_id))
      output[user][domain] = total_time_spent
  return output

In [16]:
_ = get_user_to_domain_to_total_time_spent_predicted()

In [24]:
def get_performance_for_classifier(classifier_num):
  predictions_csv = csv.reader(sdir_open('domainclass_cpn_v' + str(classifier_num) + '_randomforest_v1_predictions.csv'))
  predictions_header = next(predictions_csv)
  assert predictions_header[0] == 'predict'

  test_data_csv = csv.reader(sdir_open('domainclass_cpn_test_all_withdomain_v' + str(classifier_num) + '.csv'))
  test_data_header = next(test_data_csv)
  assert test_data_header[0] == 'time_sec'
  assert test_data_header[1] == 'user'
  assert test_data_header[2] == 'ref_domain'
  assert test_data_header[3] == 'label'

  user_to_time_to_domain_id = {}
  user_to_time_to_nextdomain_id = {}
  user_to_time_to_prev7_domains_id = {}
  user_to_time_to_immediate_prev7_domains_id = {}

  total_correct = 0
  total_items = 0

  for predictions_line,test_data_line in izip(predictions_csv, test_data_csv):
    total_items += 1
    predict = predictions_line[0]
    label = test_data_line[3]
    if predict == label:
      total_correct += 1
      continue
    time_sec = test_data_line[0] # needs to remain string so we can index into things
    user = test_data_line[1]
    ref_domain = test_data_line[2]
    ref_domain_id = domain_to_id(ref_domain)
    time_to_domain_id = user_to_time_to_domain_id.get(user, None)
    if time_to_domain_id == None:
      time_to_domain_id = get_recent_domain_id_at_seconds_for_user(user)
      user_to_time_to_domain_id[user] = time_to_domain_id
    time_to_nextdomain_id = user_to_time_to_nextdomain_id.get(user, None)
    if time_to_nextdomain_id == None:
      time_to_nextdomain_id = get_next_domain_id_at_seconds_for_user(user)
      user_to_time_to_nextdomain_id[user] = time_to_nextdomain_id
    time_to_prev7_domains_id = user_to_time_to_prev7_domains_id.get(user, None)
    if time_to_prev7_domains_id == None:
      time_to_prev7_domains_id = get_prev7_domains_id_at_seconds_for_user(user)
      user_to_time_to_prev7_domains_id[user] = time_to_prev7_domains_id
    time_to_immediate_prev7_domains_id = user_to_time_to_prev7_domains_id.get(user, None)
    if time_to_immediate_prev7_domains_id == None:
      time_to_immediate_prev7_domains_id = get_immediate_prev7_domains_id_at_seconds_for_user(user)
      user_to_time_to_immediate_prev7_domains_id[user] = time_to_immediate_prev7_domains_id
    pred_domain_id = -1
    if predict == 'c':
      pred_domain_id = time_to_domain_id[time_sec]
    elif predict == 'n':
      pred_domain_id = time_to_nextdomain_id[time_sec]
    elif predict[0] == 'p':
      num = int(predict[1])
      pred_domain_id = time_to_prev7_domains_id[time_sec][num]
    elif predict[0] == 'i':
      num = int(predict[1])
      pred_domain_id = time_to_immediate_prev7_domains_id[time_sec][num]
    if pred_domain_id == -1:
      pred_domain_id = time_to_domain_id[time_sec]
    if pred_domain_id == ref_domain_id:
      total_correct += 1

  print classifier_num, float(total_correct) / total_items, total_correct, total_items

In [16]:
for classifier_num in [89, 88, 87, 86, 85, 84, 81, 80, 79, 78, 77]:
  get_performance_for_classifier(classifier_num)

89 0.759071459932 10503101 13836775
88 0.75944871547 10508321 13836775
87 0.759371963481 10507259 13836775
86 0.758589772545 10496436 13836775
85 0.756177866591 10463063 13836775
84 0.726540902775 10052983 13836775
81 0.757509463007 10481488 13836775
80 0.759651942017 10511133 13836775
79 0.760452273019 10522207 13836775
78 0.760959327589 10529223 13836775
77 0.758549589771 10495880 13836775


10503101
13836775
0.759071459932


In [13]:
#print predictions['predict'][0,:][0,0]

In [14]:
'''
for x in glob(sdir_path('domainclass_cpn_v*.h2o')):
  print x
  #classifier = load_h2o_model('domainclass_cpn_v88_randomforest_v1.h2o')
  classifier = load_h2o_model(x)
  print get_correct_ratio_for_classifier(classifier)
'''

"\nfor x in glob(sdir_path('domainclass_cpn_v*.h2o')):\n  print x\n  #classifier = load_h2o_model('domainclass_cpn_v88_randomforest_v1.h2o')\n  classifier = load_h2o_model(x)\n  print get_correct_ratio_for_classifier(classifier)\n"